In [1]:
import mlflow
import mlflow.keras
import numpy as np
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.metrics import Recall, AUC
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import itertools
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping
import mlflow.keras
from mlflow.models import infer_signature

In [3]:
metadata = pd.read_csv('dataset/balanced_metadata.csv')
image_folder = 'dataset/balanced_dataset/'
images = []
labels = []
for i, row in metadata.iterrows():
    img_path = os.path.join(image_folder, row['image_name'])
    img = load_img(img_path, target_size=(128, 128))
    images.append(img_to_array(img))
    labels.append(row['target'])

images = np.array(images) / 255.0  # Normalisation
labels = np.array(labels)
print(labels.shape)
print(labels[:10])

(8000,)
[3 3 3 3 3 3 3 3 3 3]


In [4]:
labels = to_categorical(labels, num_classes=4)
print(images.shape)  
print(labels.shape)

(8000, 128, 128, 3)
(8000, 4)


In [5]:
def build_cnn_model(input_shape=(128, 128, 3), num_classes=4, optimizer='adam',
                    dropout_rate=0.5, activation='relu', filters=32, kernel_size=(3, 3)):
    model = Sequential()

    # Première couche convolutionnelle
    model.add(Conv2D(filters, kernel_size, activation=activation, input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    # Deuxième couche convolutionnelle
    model.add(Conv2D(filters * 2, kernel_size, activation=activation))
    model.add(MaxPooling2D((2, 2)))

    # Troisième couche convolutionnelle
    model.add(Conv2D(filters * 4, kernel_size, activation=activation))
    model.add(MaxPooling2D((2, 2)))

    # Aplatir les résultats des couches précédentes
    model.add(Flatten())

    # Couches entièrement connectées
    model.add(Dense(128, activation=activation))
    model.add(Dropout(dropout_rate))  # Dropout pour éviter l'overfitting
    model.add(Dense(num_classes, activation='softmax'))  # Classification multi-classe

    # Choisir l'optimiseur basé sur l'argument
    if optimizer == 'adam':
        optimizer_instance = Adam()
    elif optimizer == 'sgd':
        optimizer_instance = SGD()
    elif optimizer == 'rmsprop':
        optimizer_instance = RMSprop()

    # Compiler le modèle avec les métriques supplémentaires
    model.compile(optimizer=optimizer_instance, 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', Recall(), AUC()])

    return model

In [6]:
# Choix des paramètres
param_grid = {
    'optimizer': 'adam',  # Optimiseur à tester
    'dropout_rate': [0.3, 0.5],  # Taux de dropout
    'activation': 'relu',  # Fonction d'activation des couches
    'filters': [16, 32],  # Nombre de filtres dans les couches convolutionnelles
    'kernel_size': (3, 3),  # Taille des noyaux de convolution
    'batch_size': 32,  # Taille des mini-batchs
    'epochs': 10  # Nombre d'époques
}

In [7]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

- run on terminal **mlflow server --host 127.0.0.1 --port 5000**

In [8]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

- run on terminal **mlflow ui**

In [9]:
mlflow.set_experiment("Brain_Tumor_Classification")

<Experiment: artifact_location='mlflow-artifacts:/344762819335167390', creation_time=1733865490229, experiment_id='344762819335167390', last_update_time=1733865490229, lifecycle_stage='active', name='Brain_Tumor_Classification', tags={}>

In [10]:
experiment = mlflow.get_experiment_by_name("Brain_Tumor_Classification")
experiment_id = experiment.experiment_id

print(f"Experiment ID: {experiment_id}")

Experiment ID: 344762819335167390


In [11]:
# Paramètres pour la validation croisée
num_folds = 3
cv = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

In [12]:
results = []
nb = 0
for filters in param_grid['filters']:
    for dropout_rate in param_grid['dropout_rate']:
        nb = nb + 1

        # Paramètres du modèle
        params = {
            'filters': filters,
            'kernel_size': param_grid['kernel_size'],
            'dropout_rate': dropout_rate,
            'batch_size': param_grid['batch_size'],
            'epochs': param_grid['epochs'],
            'optimizer': param_grid['optimizer']
        }
        print(f"Starting experiment {nb} with parameters: {params}")

        with mlflow.start_run() as run:
            mlflow.set_tag("mlflow.runName", f"CNN : Experiment {nb}")
            mlflow.set_tag("Experiment Info", f"Experiment {nb} for NN with parameters : {params}")
            mlflow.log_params(params)

            fold_metrics = []
            for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, np.argmax(y_train, axis=1))):
                print(f"Fold {fold + 1}/{num_folds}")

                X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
                y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

                model = build_cnn_model(
                    input_shape=(128, 128, 3),
                    num_classes=4,
                    optimizer=params['optimizer'],
                    dropout_rate=params['dropout_rate'],
                    activation='relu',
                    filters=params['filters'],
                    kernel_size=params['kernel_size']
                )
                early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

                history = model.fit(
                    X_fold_train, y_fold_train,
                    validation_data=(X_fold_val, y_fold_val),
                    batch_size=params['batch_size'],
                    epochs=params['epochs'],
                    callbacks=[early_stopping],
                    verbose=1
                )

                # Prédictions et métriques sur le fold
                y_val_pred = model.predict(X_fold_val)
                y_val_pred_classes = np.argmax(y_val_pred, axis=1)
                y_val_actual_classes = np.argmax(y_fold_val, axis=1)

                accuracy = accuracy_score(y_val_actual_classes, y_val_pred_classes)
                precision = precision_score(y_val_actual_classes, y_val_pred_classes, average='weighted')
                recall = recall_score(y_val_actual_classes, y_val_pred_classes, average='weighted')
                f1 = f1_score(y_val_actual_classes, y_val_pred_classes, average='weighted')
                auc_roc = roc_auc_score(y_fold_val, y_val_pred, multi_class='ovr')

                fold_metrics.append({
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'auc_roc': auc_roc
                })

                # Logging des métriques pour chaque fold
                mlflow.log_metric(f"fold_{fold + 1}_accuracy", accuracy)
                mlflow.log_metric(f"fold_{fold + 1}_precision", precision)
                mlflow.log_metric(f"fold_{fold + 1}_recall", recall)
                mlflow.log_metric(f"fold_{fold + 1}_f1", f1)
                mlflow.log_metric(f"fold_{fold + 1}_auc_roc", auc_roc)

            # Moyennes des métriques sur tous les folds
            avg_accuracy = np.mean([m['accuracy'] for m in fold_metrics])
            avg_precision = np.mean([m['precision'] for m in fold_metrics])
            avg_recall = np.mean([m['recall'] for m in fold_metrics])
            avg_f1 = np.mean([m['f1'] for m in fold_metrics])
            avg_auc_roc = np.mean([m['auc_roc'] for m in fold_metrics])

            mlflow.log_metric("avg_accuracy", avg_accuracy)
            mlflow.log_metric("avg_precision", avg_precision)
            mlflow.log_metric("avg_recall", avg_recall)
            mlflow.log_metric("avg_f1", avg_f1)
            mlflow.log_metric("avg_auc_roc", avg_auc_roc)

            # Prédictions finales sur X_test
            y_test_pred = model.predict(X_test)
            y_test_pred_classes = np.argmax(y_test_pred, axis=1)
            y_test_actual_classes = np.argmax(y_test, axis=1)

            # Calcul des métriques sur le test
            test_accuracy = accuracy_score(y_test_actual_classes, y_test_pred_classes)
            test_precision = precision_score(y_test_actual_classes, y_test_pred_classes, average='weighted')
            test_recall = recall_score(y_test_actual_classes, y_test_pred_classes, average='weighted')
            test_f1 = f1_score(y_test_actual_classes, y_test_pred_classes, average='weighted')

            mlflow.log_metric("test_accuracy", test_accuracy)
            mlflow.log_metric("test_precision", test_precision)
            mlflow.log_metric("test_recall", test_recall)
            mlflow.log_metric("test_f1", test_f1)

            predictions_df = pd.DataFrame({"Actual": y_test_actual_classes, "Predicted": y_test_pred_classes})
            predictions_csv_path = "tmp/predictions.csv"
            predictions_df.to_csv(predictions_csv_path, index=False)
            mlflow.log_artifact(predictions_csv_path, artifact_path="predictions")

            # Matrice de confusion
            matrix = confusion_matrix(y_test_actual_classes, y_test_pred_classes)
            plt.figure(figsize=(8, 6))
            sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues')
            plt.title("Confusion Matrix")
            cf_matrix_path = "tmp/confusion_matrix.png"
            plt.savefig(cf_matrix_path)
            plt.close()
            mlflow.log_artifact(cf_matrix_path, artifact_path="cf_matrix")

            # Courbe ROC AUC
            fpr, tpr, _ = roc_curve(y_test.ravel(), y_test_pred.ravel())
            roc_auc = auc(fpr, tpr)

            plt.figure()
            plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
            plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate")
            plt.title("Receiver Operating Characteristic")
            plt.legend(loc="lower right")
            roc_curve_path = "tmp/roc_curve.png"
            plt.savefig(roc_curve_path)
            plt.close()
            mlflow.log_artifact(roc_curve_path, artifact_path="roc_curve")

            # Enregistrement du modèle final
            mlflow.keras.log_model(model, "cnn_model")
            signature = infer_signature(X_train, model.predict(X_train))
            model_info = mlflow.sklearn.log_model(
                sk_model=model,
                artifact_path="cnn_model",
                signature=signature,
                input_example=X_train[:1],
                registered_model_name="convolutional-neural-network",
            )


Starting experiment 1 with parameters: {'filters': 16, 'kernel_size': (3, 3), 'dropout_rate': 0.3, 'batch_size': 32, 'epochs': 10, 'optimizer': 'adam'}
Fold 1/3


c:\Users\user\Documents\GitHub\ML-Excuses\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 82ms/step - accuracy: 0.5318 - auc: 0.7942 - loss: 1.0487 - recall: 0.3095 - val_accuracy: 0.7762 - val_auc: 0.9451 - val_loss: 0.5673 - val_recall: 0.7299
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 9s 85ms/step - accuracy: 0.7730 - auc: 0.9417 - loss: 0.5786 - recall: 0.7306 - val_accuracy: 0.8055 - val_auc: 0.9612 - val_loss: 0.4692 - val_recall: 0.7774
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 9s 82ms/step - accuracy: 0.8371 - auc: 0.9688 - loss: 0.4223 - recall: 0.8109 - val_accuracy: 0.8360 - val_auc: 0.9644 - val_loss: 0.4480 - val_recall: 0.8248
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 76ms/step - accuracy: 0.8721 - auc: 0.9808 - loss: 0.3239 - recall: 0.8627 - val_accuracy: 0.8606 - val_auc: 0.9706 - val_loss: 0.3974 - val_recall: 0.8500
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 76ms/step - accuracy: 0.9072 - auc: 0.9883 - loss: 0.2510 - recall: 0.8951 - val_accuracy: 0.8822 - val_auc: 0.9820 - val_loss: 0.3088 - val_recall: 0.8746
Epoc

c:\Users\user\Documents\GitHub\ML-Excuses\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 14s 114ms/step - accuracy: 0.5318 - auc_1: 0.8128 - loss: 0.9981 - recall_1: 0.3503 - val_accuracy: 0.7844 - val_auc_1: 0.9517 - val_loss: 0.5251 - val_recall_1: 0.7540
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 12s 110ms/step - accuracy: 0.7882 - auc_1: 0.9493 - loss: 0.5373 - recall_1: 0.7470 - val_accuracy: 0.7698 - val_auc_1: 0.9420 - val_loss: 0.5850 - val_recall_1: 0.7481
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 12s 114ms/step - accuracy: 0.8519 - auc_1: 0.9708 - loss: 0.4067 - recall_1: 0.8310 - val_accuracy: 0.7879 - val_auc_1: 0.9598 - val_loss: 0.4720 - val_recall_1: 0.7715
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 12s 114ms/step - accuracy: 0.8896 - auc_1: 0.9846 - loss: 0.2915 - recall_1: 0.8761 - val_accuracy: 0.8231 - val_auc_1: 0.9621 - val_loss: 0.4697 - val_recall_1: 0.8120
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 12s 113ms/step - accuracy: 0.9185 - auc_1: 0.9901 - loss: 0.2293 - recall_1: 0.9121 - val_accuracy: 0.8594 - val_auc_1: 0.97

...

160/160 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


Registered model 'convolutional-neural-network' already exists. Creating a new version of this model...
2024/12/10 23:47:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: convolutional-neural-network, version 4


🏃 View run CNN : Experiment 4 at: http://127.0.0.1:5000/#/experiments/344762819335167390/runs/7b6ab6dae00443878f8e89536fcb2bd1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/344762819335167390


Created version '4' of model 'convolutional-neural-network'.


In [15]:
runs = mlflow.search_runs(experiment_ids=experiment_id)

    # Trier les exécutions par la métrique de performance choisie (par exemple, avg_accuracy)
best_cnn = runs.sort_values(by="metrics.avg_accuracy", ascending=False).iloc[0]

    # Récupérer l'ID de la meilleure exécution
best_cnn_id = best_cnn["run_id"]

print(f"Best Run ID: {best_cnn_id}")
print(f"Best Run Metrics: {best_cnn[['metrics.avg_accuracy', 'metrics.avg_f1']]}")
# Charger le modèle associé à la meilleure exécution
best_cnn_uri = f"runs:/{best_cnn_id}/cnn_model"

best_cnn_uri

Best Run ID: 7b6ab6dae00443878f8e89536fcb2bd1
Best Run Metrics: metrics.avg_accuracy    0.911528
metrics.avg_f1          0.911026
Name: 0, dtype: object


'runs:/7b6ab6dae00443878f8e89536fcb2bd1/cnn_model'

In [16]:
mlflow.register_model(
        model_uri=best_cnn_uri,
        name="BrainTumor_BestCNN"
    )

Successfully registered model 'BrainTumor_BestCNN'.
2024/12/11 00:19:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BrainTumor_BestCNN, version 1
Created version '1' of model 'BrainTumor_BestCNN'.


<ModelVersion: aliases=[], creation_timestamp=1733872740851, current_stage='None', description='', last_updated_timestamp=1733872740851, name='BrainTumor_BestCNN', run_id='7b6ab6dae00443878f8e89536fcb2bd1', run_link='', source='mlflow-artifacts:/344762819335167390/7b6ab6dae00443878f8e89536fcb2bd1/artifacts/cnn_model', status='READY', status_message='', tags={}, user_id='', version='1'>